In [1]:
import keras
import pandas as pd
import numpy as np
from html.parser import HTMLParser

Using TensorFlow backend.


In [2]:
questions = pd.read_csv('datasets/Questions.csv', encoding = 'latin1')
tags = pd.read_csv('datasets/Tags.csv', encoding = 'utf-8')

In [3]:
class HTMLStripper(HTMLParser):
    def __init__(self):
        HTMLParser.__init__(self)
        self.reset()
        self.fed = []

    def handle_data(self, d):
        self.fed.append(d)

    def handle_entityref(self, name):
        self.fed.append('&%s;' % name)

    def handle_charref(self, name):
        self.fed.append('&#%s;' % name)

    def get_data(self):
        return ''.join(self.fed)
    
    def clean_html(html):
        s = HTMLStripper()
        s.feed(html)
        return s.get_data()

In [4]:
# Combine Title with Clean Body
questions['FullText'] = questions.apply(lambda row: row.Title + ' ' + HTMLStripper.clean_html(row.Body), axis=1)

In [5]:
questions.drop(['OwnerUserId', 'CreationDate', 'Score', 'Title', 'Body'], axis=1, inplace=True)

In [6]:
questions.head()

,Id,FullText
0,6,The Two Cultures: statistics vs. machine learn...
1,21,Forecasting demographic census What are some o...
2,22,Bayesian and frequentist reasoning in plain En...
3,31,What is the meaning of p values and t values i...
4,36,Examples for teaching: Correlation does not me...


In [7]:
first_tag_for_question = tags.groupby('Id').first().reset_index().set_index('Id').to_dict()['Tag']

In [8]:
questions['Tag'] = questions.apply(lambda row: first_tag_for_question[row.Id], axis=1)

In [9]:
questions.head()

,Id,FullText,Tag
0,6,The Two Cultures: statistics vs. machine learn...,machine-learning
1,21,Forecasting demographic census What are some o...,forecasting
2,22,Bayesian and frequentist reasoning in plain En...,bayesian
3,31,What is the meaning of p values and t values i...,hypothesis-testing
4,36,Examples for teaching: Correlation does not me...,correlation


In [11]:
questions.to_csv('tagged_questions.csv', index=False)